# Daily Results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob

from datetime import datetime
import time

from common import Printable, projdir
from entrant import Entrant

import csv
import json

In [2]:
EVENTS_DIR = 'events'
CONFIG_DIR = 'config'
SESSIONS_DIR = 'config/sessions'
GPSDATA_DIR = 'gpsdata'

In [3]:
CONFIG_JSON = 'config.json'
ENTRANTS_CSV = 'entrants.csv'
COURSES_JSON = 'courses.json'

In [4]:
T_RUN = 'Run'
T_FILENAME = 'Filename'
T_START_TIME = 'Start Time'
T_DURATION = 'Duration'
T_SPEED = 'Speed (kts)'
T_COG = 'COG'

In [5]:
try:
    wsw_full_refresh = int(os.environ['WSW_FULL_REFRESH'])
except:
    wsw_full_refresh = 1

## SpeedRun Class

Class to manage runs by an entrant

In [6]:
class SpeedRun(Printable):
    def __init__(self, headers, values, verbosity=1):
        '''Initialise speed run object'''    

        super().__init__(verbosity=verbosity)

        # Store speed run details in a dictionary - makes reporting easier
        self.data = {}
        for i in range(len(values)):
            try:
                if headers[i] == T_RUN:
                    self.data[headers[i]] = int(values[i])
                elif headers[i] in [T_DURATION, T_SPEED, T_COG]:
                    self.data[headers[i]] = float(values[i])
                else:
                    self.data[headers[i]] = values[i]
            except:
                self.logError('Problem parsing run data - "{}"'.format(values))
                raise

## Course Class

Class to manage courses - start / end times

In [7]:
class Course(Printable):
    def __init__(self, session, courseId, startTime, endTime, verbosity=1):
        '''Initialise course object'''

        super().__init__(verbosity=verbosity)

        self.session = session
        self.entrants = session.entrants
        self.courseDate = session.date
        
        self.courseId = courseId
        self.startTime = startTime
        self.endTime = endTime      

        self.minStartTime = '23:59:59'
        self.maxStartTime = '00:00:00'

        self.runs = {}
        self.bestRuns = {}

        self.numRuns = 0
        

    def getEntrant(self, gt31Ids, gt31Id, gt31Serial):
        '''Get entrant from the GT-31 ID and serial'''

        if gt31Id not in gt31Ids:
            self.logWarning('Unrecognised GT-31 ID on {} - {}'.format(self.courseDate, gt31Id))

            entrantId = max(self.entrants) + 1
            entrant = Entrant(["ID", "GT31 ID", "GT31 SN"], [entrantId, gt31Id, gt31Serial], verbosity=self.verbosity)

            self.entrants[entrantId] = entrant

            gt31Ids[gt31Id] = entrant

        # TODO - check for unique serial (just in case GPS name has been changed)

        else:
            if gt31Ids[gt31Id].gt31SerialNumbers and gt31Serial not in gt31Ids[gt31Id].gt31SerialNumbers:
                self.logWarning('Unrecognised GT-31 SN for {} ({}) on {} - {} vs {}'.format(
                        gt31Ids[gt31Id].getValue('Name'), gt31Ids[gt31Id].getValue('Craft Type'),
                        self.courseDate,
                        gt31Serial, gt31Ids[gt31Id].gt31SerialNumbers))

            entrant = gt31Ids[gt31Id]
            
        return entrant


    def storeRun(self, entrantId, speedRun):
        '''Store run in memory'''

        if entrantId in self.runs:
            self.runs[entrantId].append(speedRun)

            if speedRun.data[T_SPEED] > self.bestRuns[entrantId].data[T_SPEED]:
                self.bestRuns[entrantId] = speedRun
        else:
            self.runs[entrantId] = [speedRun]
            self.bestRuns[entrantId] = speedRun

        self.session.storeRun(entrantId, speedRun)


    def loadRuns(self, csvPath, gt31Ids):
        '''Read speeds from CSV files'''

        prevGt31Id = None
        headers = [T_RUN, T_FILENAME, T_START_TIME, T_DURATION, T_SPEED, T_COG]

        with open(csvPath, 'r') as f:
            csvReader = csv.reader(f)
            for values in csvReader:
                if len(values) != len(headers):
                    raise ValueError('Incorrect number of fields in "{}" - {}'.format(os.path.basename(csvPath), values))

                # GPSResults does not output a header line
                run, filename, startTime, duration, speed, cog = values

                # Split up record and format the GT-31 details
                try:
                    gt31Id, gt31Serial, fileDate, fileTime = os.path.splitext(filename)[0].split('_')[:4]
                except:
                    self.logError('Problem parsing GT-31 details in run data - "{}"'.format(filename))
                    raise
                gt31Id = gt31Id.upper()

                # Validate each newly encountered GT-31
                if gt31Id != prevGt31Id:
                    entrant = self.getEntrant(gt31Ids, gt31Id, gt31Serial)
                    entrantId = entrant.getValue('ID')

                speedRun = SpeedRun(headers, values, verbosity=self.verbosity)

                if startTime < self.minStartTime:
                    self.minStartTime = startTime
                if startTime > self.maxStartTime:
                    self.maxStartTime = startTime

                if startTime >= self.startTime and startTime <= self.endTime:
                    self.storeRun(entrantId, speedRun)
                    self.numRuns += 1

                prevGt31Id = gt31Id

            self.logInfo('{} runs by {} participants down course {} on {} - {} to {}'.format(
                self.numRuns, len(self.runs), self.courseId, self.courseDate, self.minStartTime, self.maxStartTime))
            
        if self.minStartTime < self.startTime:
            self.logWarning('Runs found before course {} opened on {} - earliest was {}'.format(
                self.courseId, self.courseDate, self.minStartTime))

        if self.maxStartTime > self.endTime:
            self.logWarning('Runs found after course {} closed on {} - latest was {}'.format(
                self.courseId, self.courseDate, self.maxStartTime))

## Session Class

Class to manage sessions

In [8]:
class Session(Printable):
    def __init__(self, event, dataPath, configPath, verbosity=1):
        '''Initialise session object'''

        super().__init__(verbosity=verbosity)

        self.event = event
        self.entrants = event.entrants

        self.dataPath = dataPath
        self.configPath = configPath

        self.date = os.path.basename(configPath)
        self.year = self.date[:4]

        self.courses = {}
        self.runs = {}
        self.bestRuns = {}


    def loadConfig(self):
        '''Read config from JSON'''

        filename = os.path.join(self.configPath, COURSES_JSON)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            courseDict = json.loads(jsonTxt)
            
            for courseId in courseDict:
                # Course ID should be uppercase but just to be sure...
                courseId = courseId.upper()
                if courseId in self.courses:
                    raise ValueError('Duplicate course "{}" for {}'.format(courseId, self.date))
                course = Course(self, courseId,
                                courseDict[courseId]['Start Time'], courseDict[courseId]['End Time'],
                                verbosity=self.verbosity)
                self.courses[courseId] = course


    def loadRuns(self, gt31Ids):
        '''Read speeds from CSV files'''

        csvPaths = sorted(glob.glob(os.path.join(self.dataPath, '*')))
        
        for csvPath in csvPaths:
            prefix, sessionDate, courseId = os.path.splitext(os.path.basename(csvPath))[0].split('_')
            suffix = os.path.splitext(csvPath)[1]

            # Verify the filename
            if suffix.lower() != '.csv':
                raise ValueError('Invalid file suffix "{}" for {}'.format(suffix, os.path.basename(csvPath)))
            if prefix.upper() != 'GPSDATA':
                raise ValueError('Invalid file prefix "{}" for {}'.format(prefix, os.path.basename(csvPath)))
            if sessionDate != self.date:
                raise ValueError('Invalid file date "{}" for {}'.format(sessionDate, os.path.basename(csvPath)))
            if courseId.upper() not in self.courses:
                raise ValueError('Invalid course "{}" for {}'.format(courseId, os.path.basename(csvPath)))
            
            course = self.courses[courseId.upper()]
            course.loadRuns(csvPath, gt31Ids)


    def storeRun(self, entrantId, speedRun):
        '''Store run in memory'''

        if entrantId in self.runs:
            self.runs[entrantId].append(speedRun)

            if speedRun.data[T_SPEED] > self.bestRuns[entrantId].data[T_SPEED]:
                self.bestRuns[entrantId] = speedRun
        else:
            self.runs[entrantId] = [speedRun]
            self.bestRuns[entrantId] = speedRun
            
        self.event.storeRun(entrantId, speedRun)

## Event Class

Class to manage events

In [9]:
class Event(Printable):
    def __init__(self, path, verbosity=1):
        
        super().__init__(verbosity=verbosity)

        self.path = path
        self.year = int(os.path.basename(path))

        self.entrants = {}
        self.gt31Ids = {}       

        self.sessions = {}
        self.runs = {}
        self.bestRuns = {}


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r', encoding='utf-8') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(headers, values, verbosity=self.verbosity)
                if entrant.getValue('ID') not in self.entrants:
                    self.entrants[entrant.getValue('ID')] = entrant
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        self.logInfo('{} entrants loaded'.format(len(self.entrants)))


    def indexGt31s(self):
        '''Create GT-31 indices for entrants'''

        for entrantId in self.entrants:
            entrant = self.entrants[entrantId]

            for gt31Id in entrant.gt31Ids:
                self.gt31Ids[gt31Id.upper()] = entrant
        

    def summariseEntrants(self):
        '''Print summary of entrants'''
        
        pass


    def loadSessions(self):
        '''Load all of the event sessions - absence of config will result in an exception (intentional)'''
        
        dataPaths = sorted(glob.glob(os.path.join(self.path, GPSDATA_DIR, '20[0-9][0-9][0-1][0-9][0-3][0-9]')))
        
        for dataPath in dataPaths:
            configPath = os.path.join(self.path, SESSIONS_DIR, os.path.basename(dataPath))
            session = Session(self, dataPath, configPath, verbosity=self.verbosity)

            session.loadConfig()
            session.loadRuns(self.gt31Ids)
            # session.processRuns()

            self.sessions[session.date] = session
            

    def processEvent(self):
        '''Read entrants from config folder'''

        print('Processing {}...'.format(self.year))

        self.loadEntrants()  
        self.indexGt31s()
        self.loadSessions()

        print('All done!\n')


    def storeRun(self, entrantId, speedRun):
        '''Store run in memory'''

        if entrantId in self.runs:
            self.runs[entrantId].append(speedRun)

            if speedRun.data[T_SPEED] > self.bestRuns[entrantId].data[T_SPEED]:
                self.bestRuns[entrantId] = speedRun
        else:
            self.runs[entrantId] = [speedRun]
            self.bestRuns[entrantId] = speedRun

## Process Years

Process all available years

In [10]:
pc1 = time.perf_counter()

In [11]:
# Read main config
filename = os.path.join(projdir, CONFIG_DIR, CONFIG_JSON)
with open(filename, 'r', encoding='utf-8') as f:
    jsonTxt = f.read()
    config = json.loads(jsonTxt)

In [12]:
# Process all events - 2010 onwards
eventPaths = sorted(glob.glob(os.path.join(projdir, EVENTS_DIR, '20[0-9][0-9]')))
year = datetime.now().year
for eventPath in eventPaths:
    if int(os.path.basename(eventPath)) <= year:
        if int(os.path.basename(eventPath)) == year or wsw_full_refresh:
            event = Event(eventPath, verbosity=1)
            event.processEvent()

Processing 2010...
All done!

Processing 2011...
All done!

Processing 2012...
All done!

Processing 2013...
All done!

Processing 2014...
All done!

Processing 2015...
All done!

Processing 2016...
All done!

Processing 2017...
All done!

Processing 2018...
All done!

Processing 2019...
All done!

Processing 2021...
All done!



In [13]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 0.75 seconds


## All Done!